# Multimodal RAG with LlamaIndex

This cookbook shows how to perform RAG on the table and text extraction output of nv-ingest's pdf extraction tools using LlamaIndex

To start we'll need to make sure we have llama_index installed

In [ ]:
pip install -qU llama_index llama-index-embeddings-nvidia llama-index-llms-nvidia

Then, we'll use nv-ingest to parse an example pdf that contains text, tables, charts, and images. We'll need to make sure to have the nv-ingest microservice up and running at localhost:7670 along with the supporting NIMs. To do this, follow the nv-ingest [quickstart guide](https://github.com/NVIDIA/nv-ingest?tab=readme-ov-file#quickstart). Once the microservice is ready we can create a job with the nv-ingest python client

In [1]:
from nv_ingest_client.client import NvIngestClient
from nv_ingest_client.message_clients.rest.rest_client import RestClient
from nv_ingest_client.primitives import JobSpec
from nv_ingest_client.primitives.tasks import ExtractTask


from nv_ingest_client.util.file_processing.extract import extract_file_content
import logging, time

logger = logging.getLogger("nv_ingest_client")

file_name = "../data/multimodal_test.pdf"
file_content, file_type = extract_file_content(file_name)

job_spec = JobSpec(
    document_type=file_type,
    payload=file_content,
    source_id=file_name,
    source_name=file_name,
    extended_options={
        "tracing_options": {
            "trace": True,
            "ts_send": time.time_ns()
        }
    },
)

And then, we can and submit a task to extract the text and tables from the example pdf

In [2]:
extract_task = ExtractTask(
    document_type=file_type,
    extract_text=True,
    extract_images=False,
    extract_tables=True,
)


job_spec.add_task(extract_task)

client = NvIngestClient(
  message_client_hostname="localhost",
  message_client_port=7670
)

job_id = client.add_job(job_spec)

client.submit_job(job_id, "morpheus_task_queue")

result = client.fetch_job_result(job_id, timeout=60)

In [3]:
result[0][0][0]

{'document_type': 'text',
 'metadata': {'content': 'TestingDocument\r\nA sample document with headings and placeholder text\r\nIntroduction\r\nThis is a placeholder document that can be used for any purpose. It contains some \r\nheadings and some placeholder text to fill the space. The text is not important and contains \r\nno real value, but it is useful for testing. Below, we will have some simple tables and charts \r\nthat we can use to confirm Ingest is working as expected.\r\nTable 1\r\nThis table describes some animals, and some activities they might be doing in specific \r\nlocations.\r\nAnimal Activity Place\r\nGira@e Driving a car At the beach\r\nLion Putting on sunscreen At the park\r\nCat Jumping onto a laptop In a home o@ice\r\nDog Chasing a squirrel In the front yard\r\nChart 1\r\nThis chart shows some gadgets, and some very fictitious costs. Section One\r\nThis is the first section of the document. It has some more placeholder text to show how \r\nthe document looks like.

Now, we have the extraction results in the nv-ingest metadata format. We'll separate the content out of this and load it into LlamaIndex documents

In [4]:
from llama_index.core import Document

texts = []
tables = []
for element in result[0][0]:
    if element['document_type'] == 'text':
        texts.append(Document(text=element['metadata']['content']))
    elif element['document_type'] == 'structured':
        tables.append(Document(text=element['metadata']['table_metadata']['table_content']))

In [5]:
texts

[Document(id_='2b6bac68-5e1e-4a5a-a1ff-ef78b00ca711', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='TestingDocument\r\nA sample document with headings and placeholder text\r\nIntroduction\r\nThis is a placeholder document that can be used for any purpose. It contains some \r\nheadings and some placeholder text to fill the space. The text is not important and contains \r\nno real value, but it is useful for testing. Below, we will have some simple tables and charts \r\nthat we can use to confirm Ingest is working as expected.\r\nTable 1\r\nThis table describes some animals, and some activities they might be doing in specific \r\nlocations.\r\nAnimal Activity Place\r\nGira@e Driving a car At the beach\r\nLion Putting on sunscreen At the park\r\nCat Jumping onto a laptop In a home o@ice\r\nDog Chasing a squirrel In the front yard\r\nChart 1\r\nThis chart shows some gadgets, and some very fictitious costs. Section One\r

In [6]:
tables

[Document(id_='0d2024b3-c41a-4604-9779-e83106d07d78', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='locations. Animal Activity Place Giraffe Driving a car At the beach Lion Putting on sunscreen At the park Cat Jumping onto a laptop In a home office Dog Chasing a squirrel In the front yard', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='ccfba084-b81d-4319-be0d-573bc5742d45', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This chart shows some gadgets, and some very fictitious costs. >\\n7938.758 ext. Print & Maroon Bookshelf Fine Art Poems Collection dla Cemicon Diamtháhn | Gadgets and their cost\nSollywood for Coasters | 19875.075     t158.281 \n Hammer | 19871.55 \n Powerdrill | 12044.625 \n Bluetooth speaker 

Now, the text and table content is ready to be embedded and stored. We'll set our NVIDIA api key in order to use an embedding model from NVIDIA's API catalog

In [7]:
import os
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.nvidia import NVIDIAEmbedding

# TODO: Add your NVIDIA API key here
os.environ["NVIDIA_API_KEY"] = "<YOUR_NVIDIA_API_KEY>"

embed_model = NVIDIAEmbedding(model="NV-Embed-QA")
index = VectorStoreIndex.from_documents(texts+tables, embed_model=embed_model)

Next, we'll use our vectorstore to create a query engine that handles the RAG pipeline and we'll use an llm from the NVIDIA API catalog to generate the final response

In [30]:
from llama_index.llms.nvidia import NVIDIA

llm = NVIDIA(model="meta/llama-3.1-405b-instruct")
query_engine = index.as_query_engine(llm=llm)

And finally, we can ask it questions about our example PDF

In [9]:
query_engine.query("What is the dog doing and where?").response

'The dog is chasing a squirrel in the front yard.'

## Milvus

Alternatively, we can use the embedding NIM and the milvus vector database packaged with NV-Ingest. This requires pymilvus and the embedding, milvus, etcd, attu, and minio microservices to be up and [running](https://github.com/NVIDIA/nv-ingest/blob/main/docs/deployment.md#launch-nv-ingest-micro-services). This has the benefit of directly sending the extraction results to the embedding microservice and then to a vector database without roundtripping between the client and the NV-Ingest microservice between each step

In [ ]:
pip install -qU pymilvus llama-index-vector-stores-milvus

First, we'll create a new job spec

In [11]:
file_name = "../data/multimodal_test.pdf"
file_content, file_type = extract_file_content(file_name)

job_spec = JobSpec(
    document_type=file_type,
    payload=file_content,
    source_id=file_name,
    source_name=file_name,
    extended_options={
        "tracing_options": {
            "trace": True,
            "ts_send": time.time_ns()
        }
    },
)

Then, we'll add the extraction task again, but this time we'll also add an embed task and a vector database upload task

In [12]:
from nv_ingest_client.primitives.tasks import EmbedTask
from nv_ingest_client.primitives.tasks import VdbUploadTask

extract_task = ExtractTask(
    document_type=file_type,
    extract_text=True,
    extract_images=False,
    extract_tables=True,
)

embed_task = EmbedTask(
    text=True,
    tables=True,
)

vdb_upload_task = VdbUploadTask()

job_spec.add_task(extract_task)
job_spec.add_task(embed_task)
job_spec.add_task(vdb_upload_task)

client = NvIngestClient(
  message_client_hostname="localhost",
  message_client_port=7670
)

job_id = client.add_job(job_spec)

client.submit_job(job_id, "morpheus_task_queue")

result = client.fetch_job_result(job_id, timeout=60)

Next, we'll connect LlamaIndex to our collection in Milvus and create a new query engine from it

In [27]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.milvus import MilvusVectorStore


vector_store = MilvusVectorStore(
    uri="http://localhost:19530",
    collection_name="nv_ingest_collection",
    doc_id_field="pk",
    embedding_field="vector",
    text_key="text",
    dim=1024,
    overwrite=False
)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, embed_model=embed_model)
query_engine = index.as_query_engine(llm=llm)

And then finally, we can query our Milvus vector database just as we did before with Chroma

In [29]:
query_engine.query("What is the dog doing and where?").response

'The dog is chasing a squirrel in the front yard.'